In [5]:
!pip3 install opencv-python
!pip install scikit-learn
!pip install tensorflow
!pip install --upgrade numpy

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.1 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.1 which is incompatible.


  Using cached numpy-2.3.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.1-cp311-cp311-win_amd64.whl (13.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


ImportError: cannot import name '_slice' from 'numpy._core.umath' (d:\College\dl\mask detector\.venv\Lib\site-packages\numpy\_core\umath.py)

In [ ]:
train_dir = 'test'
img_size = (150, 150)

# Load images with OpenCV
def load_images_from_folder(folder):
    images = []
    labels = []
    class_names = sorted(os.listdir(folder))  # Ensure consistent label order

    for label, class_name in enumerate(class_names):
        class_path = os.path.join(folder, class_name)
        if not os.path.isdir(class_path):
            continue
        for filename in os.listdir(class_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_path, filename)
                img = cv2.imread(img_path)
                if img is None:
                    continue  # Skip corrupted files
                img = cv2.resize(img, img_size)
                img = img / 255.0  # Normalize
                images.append(img)
                labels.append(label)

    return np.array(images), np.array(labels)

# Load data
X, y = load_images_from_folder(train_dir)


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Build CNN
cnn = Sequential([
    Conv2D(100, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(100, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dropout(0.5),

    Dense(50, activation='relu'),
    Dense(35, activation='relu'),

    Dense(1, activation='sigmoid')
])

# Compile model
cnn.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = cnn.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)

# Save model
cnn.save("mask_detector_model.h5")

In [ ]:
cnn = load_model('mask_detector_model.h5')
labels_dict = {0: 'No mask', 1: 'Mask'}
color_dict  = {0: (0, 0, 255), 1: (0, 255, 0)}

imgsize    = 4
camera     = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

while True:
    ret, frame = camera.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    small = cv2.resize(frame, (frame.shape[1]//imgsize, frame.shape[0]//imgsize))
    faces = face_cascade.detectMultiScale(small)

    for (x0, y0, w0, h0) in faces:
        x, y, w, h = [v * imgsize for v in (x0, y0, w0, h0)]
        face = frame[y:y+h, x:x+w]
        if face.size == 0:
            continue

        # preprocess for model
        face_resized = cv2.resize(face, (150, 150))
        face_norm    = face_resized.astype('float32') / 255.0
        face_input   = np.expand_dims(face_norm, axis=0)

        # predict
        score = float(cnn.predict(face_input)[0][0])  
        label = 1 if score >= 0.5 else 0

        # draw
        color = color_dict[label]
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.rectangle(frame, (x, y-30), (x+w, y), color, -1)
        text = f"{labels_dict[label]}: {score:.2f}"
        cv2.putText(frame, text, (x+5, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    cv2.imshow("LIVE MASK DETECTION", frame)
    if cv2.waitKey(10) == 27:  # ESC
        break

camera.release()
cv2.destroyAllWindows()